In [2]:
"Modified version of Jesse's ESDG linear advection code to use his StartUpDG library"

using Revise # reduce need for recompile
using Plots
using LinearAlgebra

using StartUpDG

"Approximation parameters"
p   = 3 # The order of approximation
N_el_1D = 4
CFL = .5
T   = 2.0 # endtime

"Reference element data"
elem_data = RefElemData(Tri(),p)

"Mesh related variables"
VXY, EToV = uniform_mesh(Tri(),N_el_1D)
mesh_data = MeshData(VXY, EtoV, elem_data)

"Construct global coordinates"
r1,s1 = nodes_2D(1)
V1 = vandermonde_2D(1,r,s)/vandermonde_2D(1,r1,s1)

"Make periodic"
LX,LY = (x->maximum(x)-minimum(x)).((VX,VY)) # find lengths of domain
mapPB = build_periodic_boundary_maps(xf,yf,LX,LY,Nfaces*K,mapM,mapP,mapB)
mapP[mapB] = mapPB

"Geometric factors and surface normals"
rxJ, sxJ, ryJ, syJ, J = geometric_factors(x, y, Dr, Ds)
nxJ = (Vf*rxJ).*nrJ + (Vf*sxJ).*nsJ;
nyJ = (Vf*ryJ).*nrJ + (Vf*syJ).*nsJ;
sJ = @. sqrt(nxJ^2 + nyJ^2)

"initial conditions"
u = @. exp(-25*(x^2+y^2))

"Time integration"
rk4a,rk4b,rk4c = rk45_coeffs()
CN = (N+1)*(N+2)/2  # estimated trace constant
dt = CFL * 2 / (CN*K1D)
Nsteps = convert(Int,ceil(T/dt))
dt = T/Nsteps

"pack arguments into tuples"
ops = (Dr,Ds,LIFT,Vf)
vgeo = (rxJ,sxJ,ryJ,syJ,J)
fgeo = (nxJ,nyJ,sJ)
nodemaps = (mapP,mapB)

function rhs(u,ops,vgeo,fgeo,nodemaps)
    # unpack args
    Dr,Ds,LIFT,Vf = ops
    rxJ,sxJ,ryJ,syJ,J = vgeo
    nxJ,nyJ,sJ = fgeo
    (mapP,mapB) = nodemaps

    uM = Vf*u # can replace with nodal extraction
    uP = uM[mapP]
    du = uP-uM

    ur = Dr*u
    us = Ds*u
    ux = @. rxJ*ur + sxJ*us

    tau = .5
    rhsu = ux + LIFT*(@. .5*du*nxJ - tau*du*abs(nxJ))

    return -rhsu./J
end

resu = zeros(size(x))
for i = 1:Nsteps
    global u, resu # for scoping - these variables are updated

    for INTRK = 1:5
        rhsu = rhs(u,ops,vgeo,fgeo,nodemaps)
        @. resu = rk4a[INTRK]*resu + dt*rhsu
        @. u += rk4b[INTRK]*resu
    end

    if i%10==0 || i==Nsteps
        println("Number of time steps $i out of $Nsteps")
    end
end

"plotting nodes"

gr(aspect_ratio=1,legend=false,
   markerstrokewidth=0,markersize=2)

rp, sp = equi_nodes_2D(10)
Vp = vandermonde_2D(N,rp,sp)/V
vv = Vp*u
scatter(Vp*x,Vp*y,vv,zcolor=vv,camera=(0,90))

┌ Error: Some Julia code in the VS Code extension crashed
└ @ VSCodeServer /Users/tristanmontoya/.vscode/extensions/julialang.language-julia-1.4.3/scripts/error_handler.jl:15
ERROR: MethodError: no method matching revise()
The applicable method may be too new: running in world age 29627, while current world is 29898.
Closest candidates are:
  revise(; throw) at /Users/tristanmontoya/.julia/packages/Revise/3RMhb/src/packagedef.jl:740 (method too new to be called from this world context.)
  revise(::REPL.REPLBackend) at /Users/tristanmontoya/.julia/packages/Revise/3RMhb/src/packagedef.jl:839 (method too new to be called from this world context.)
  revise(::Module) at /Users/tristanmontoya/.julia/packages/Revise/3RMhb/src/packagedef.jl:847 (method too new to be called from this world context.)
Stacktrace:
 [1] revise()
   @ VSCodeServer ~/.vscode/extensions/julialang.language-julia-1.4.3/scripts/packages/VSCodeServer/src/misc.jl:157
 [2] (::VSCodeServer.var"#98#99"{VSCodeServer.NotebookRu